In [1]:
import uuid

import pprint
import logging
import boto3

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
input_text:str = "I am looking to book a flight from Calgary to Paris on Apr 1 and my membership number is 123"
agent_id:str = 'OC9DZCN1WG'                                             # note this from the agent console on Bedrock
agent_alias_id:str = 'TSTALIASID'                                       # fixed for draft version of the agent
session_id:str = str(uuid.uuid1())                                      # random identifier
enable_trace:bool = True

In [7]:
session = boto3.Session(profile_name='bach-dev', region_name='us-east-1')
client = session.client(service_name="bedrock-agent-runtime")

logger.info(client)

[2024-03-31 08:56:02,973] p4840 {276264083.py:4} INFO - <botocore.client.AgentsforBedrockRuntime object at 0x000001B2DCB7CC10>


In [12]:
# invoke the agent API
response = client.invoke_agent(
    inputText=input_text,
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace
)

logger.info(pprint.pprint(response))

[2024-03-31 08:56:38,279] p4840 {899143767.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 31 Mar 2024 14:56:37 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ca4f8152-ef6e-11ee-af01-f02f7418bd35',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '24a30bb5-5c12-4b58-ac9c-ea3c2278e6d5'},
                      'HTTPStatusCode': 200,
                      'RequestId': '24a30bb5-5c12-4b58-ac9c-ea3c2278e6d5',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x000001B2DB01B9D0>,
 'contentType': 'application/json',
 'sessionId': 'ca4f8152-ef6e-11ee-af01-f02f7418bd35'}


In [13]:
%%time
import json
event_stream = response['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}") 
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-03-31 08:56:42,091] p4840 {<timed exec>:12} INFO - {
  "agentId": "OC9DZCN1WG",
  "agentAliasId": "TSTALIASID",
  "sessionId": "ca4f8152-ef6e-11ee-af01-f02f7418bd35",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "24a30bb5-5c12-4b58-ac9c-ea3c2278e6d5-pre-0",
        "text": "You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<tools>\n    <tool_description>\n<tool_name>GET::TravelActionGroup::getAirportCode</tool_name>\n<description>Based on the provided city name, returns the airport code closest to the city</description>\n<parameters>\n<parameter>\n<name>city</name

CPU times: total: 0 ns
Wall time: 41.5 s


In [14]:
# Response from the agent
print(agent_answer)

Here are the available flight options from Calgary to Paris on April 1st:

Flight 123: 
Departing YEG at 11:00pm on March 31, layover in YYC, departing YYC at 3:19am on April 1, arriving CDG at 11:12am on April 1.
Total price: $617.43

Flight 346:
Departing YEG at 12:15pm on March 31, layover in YYC, departing YYC at 3:19am on April 1, arriving CDG at 11:12am on April 1.  
Total price: $567.43

Would you like to book one of these flights? I can use your saved credit card and reward dollars if you want to book.
